# Подготовка

In [2]:
import pandas as pd
from pathlib import Path
import docx
from pypdf import PdfReader
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import pymorphy3

In [3]:
additional_stopwords = {"а",
"без",
"безо",
"в",
"во",
"для",
"до",
"за",
"из",
"изо",
"к",
"ко",
"на",
"над",
"надо",
"о",
"об",
"обо",
"от",
"ото",
"по",
"под",
"подо",
"при",
"про",
"с",
"со",
"у",
"через",
"чрез",
"и",
"тоже",
"также",
"но",
"однако",
"зато",
"же",
"или",
"либо",
"то",
"что",
"чтобы",
"как",
"будто",
"когда",
"пока",
"едва",
"потому",
"так",
"ибо",
"оттого",
"чтобы",
"если",
"бы",
"раз",
"коли",
"хотя",
"хоть",
"пускай",
"как",
"будто",
"словно",
"точно",
"давай",
"давайте",
"пусть",
"пускай",
"бы",
"б",
"же",
"даже",
"именно",
"только",
"лишь",
"хоть",
"исключительно",
"единственно",
"просто",
"прямо",
"вот",
"вон",
"это",
"ли",
"ль",
"разве",
"неужели",
"не",
"ни",
"так",
"точно",
"конечно",
"едва",
"только",
"всего",
"исключительно",
"где",
"куда",
"откуда",
"когда",
"зачем",
"почему",
"отчего",
"как",
"сколько",
"насколько",
"что",
"кто",
"какой",
"каков",
"который",
"чей"}

In [4]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/iversy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/iversy/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/iversy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Часть 1

In [5]:


## Часть 1

def load_csv(path: Path, sep: str = ",") -> pd.Series:
    df = pd.read_csv(path, usecols=["text"])["text"]
    return df

def load_txt(path: Path, sep: str = "SEP") -> pd.Series:
    text = path.read_text()
    parts = text.split(sep)
    parts = [p.strip() for p in parts if p]
    return pd.Series(parts, dtype="string").reset_index(drop=True)

def load_docx(path: Path, sep: str = "SEP") -> pd.Series:
    doc = docx.Document(path)
    text = ''.join(paragraph.text for paragraph in doc.paragraphs)
    parts = [p.strip() for p in text.split(sep)]
    return pd.Series(parts, dtype="string").reset_index(drop=True)

def load_pdf(path: Path, sep="SEP") -> pd.Series:
    reader = PdfReader(path)
    text = "".join(page.extract_text() for page in reader.pages)
    parts = [p.strip() for p in text.split(sep)]
    return pd.Series(parts, dtype="string").reset_index(drop=True)



def load(path: Path) -> pd.DataFrame | pd.Series:
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")

    match path.suffix:
        case ".csv":
            return load_csv(path)
        case ".txt":
            return load_txt(path)
        case ".docx":
            return load_docx(path)
        case ".pdf":
            return load_pdf(path)
        case _:
            raise ValueError
        

# Часть 2

In [6]:
def _clean(text: str) -> str:
    text = text.lower()
    text, _ = re.subn('[0-9]',"", text)
    text, _ = re.subn('[,.;:]', '', text)
    text, _ = re.subn(' +', ' ', text)
    return text

def clean(text_series: pd.Series) -> pd.Series:
    result = text_series.copy()
    result = result.apply(_clean)
    return result


# Часть 3-4


In [7]:

stemmer = SnowballStemmer('russian')

russian_stopwords = stopwords.words('russian')
all_stopwords = set( additional_stopwords)
morph = pymorphy3.MorphAnalyzer()


def flatten(x):
    for item in x:
        if isinstance(item, (list, tuple)):
            yield from flatten(item)
        else:
            yield item

def _token_word(text):
    return word_tokenize(text, language="russian")
def _token_sent(text):
    return sent_tokenize(text, language="russian")


def get_tokens_sent(series: pd.Series) -> pd.Series:
    series_parts = flatten([_token_sent(row) for row in series])
    return pd.Series(series_parts, dtype="string").reset_index(drop=True)


def get_tokens_word(series: pd.Series) -> pd.Series:
    series_parts = flatten([_token_word(row) for row in series])
    return pd.Series(series_parts, dtype="string").reset_index(drop=True)

def stem_text(tokens):
    return pd.Series([stemmer.stem(word) for word in tokens], dtype='string').reset_index(drop=True)


def lemmatize_text(tokens):
    lemmas = []
    for word in tokens:
        # Анализ слова и выбор наиболее вероятной формы
        parsed = morph.parse(word)[0]
        lemmas.append(parsed.normal_form)
    return pd.Series(lemmas, dtype="string").reset_index(drop=True)


def delete_stop(series: pd.Series) -> pd.Series:
    result = []
    for token in series:
        if not token in all_stopwords: 
            result.append(token)
    
    return pd.Series(result, dtype="string").reset_index(drop=True)

def _tokenise(series: pd.Series) -> pd.Series:
    result = get_tokens_word(series)
    result = delete_stop(result)
    result = stem_text(result)
    result = lemmatize_text(result)
    return result

In [8]:
path = "news_5k.txt"

In [9]:
path = Path(path)

serega = load(path)
print(serega)
sergei = clean(serega)
print(sergei)
sergei_sergeevich = _tokenise(sergei)
print(sergei_sergeevich)


0       В 1930-е годы Советский Союз охватила лихорадк...
1       Олимпийская чемпионка по фигурному катанию  Ал...
2       Российский врач-диетолог Римма Мойсенко объясн...
3       В 2019 году телеканал «Ю» запустил адаптацию з...
4       Актер  Михаил Ефремов  систематически употребл...
                              ...                        
4996    В Москве за сутки умерли 67 пациентов с корона...
4997    Число официально зарегистрированных безработны...
4998    28 апреля в ютьюбе прошел онлайн-митинг «За жи...
4999    Организаторы «Оскара» объявили, что в связи с ...
5000                                                     
Length: 5001, dtype: string
0       в -е годы советский союз охватила лихорадка — ...
1       олимпийская чемпионка по фигурному катанию али...
2       российский врач-диетолог римма мойсенко объясн...
3       в году телеканал «ю» запустил адаптацию знамен...
4       актер михаил ефремов систематически употреблял...
                              ...           

# Часть 5

In [10]:
def statistics(path):
    data = load(path)
    data = clean(data)
    print("-"*40)
    print(data.info())
    print("-"*40)
    print(data.describe())

    tokens_word = get_tokens_word(data)
    print("-"*40)

    print(tokens_word.info())
    print("-"*40)
    print(tokens_word.describe())
    tokens_word = tokens_word.apply(len)
    print("-"*40)

    print(tokens_word.info())
    print("-"*40)
    print(tokens_word.describe())
    tokens_sent = get_tokens_sent(data)
    print("-"*40)

    print(tokens_sent.info())
    print("-"*40)
    print(tokens_sent.describe())
    tokens_sent = tokens_sent.apply(len)

    print("-"*40)

    print(tokens_sent.info())
    print("-"*40)
    print(tokens_sent.describe())

In [14]:
p1 = Path("news_5k.txt")
p2 = Path("news_5k.docx")
p3 = Path("news_5k.pdf")
p4 = Path("news_5k.csv")

In [15]:
statistics(p1)

----------------------------------------
<class 'pandas.Series'>
RangeIndex: 5001 entries, 0 to 5000
Series name: None
Non-Null Count  Dtype
--------------  -----
5001 non-null   str  
dtypes: str(1)
memory usage: 39.2 KB
None
----------------------------------------
count                                                  5001
unique                                                 5001
top       в -е годы советский союз охватила лихорадка — ...
freq                                                      1
dtype: object
----------------------------------------
<class 'pandas.Series'>
RangeIndex: 1043877 entries, 0 to 1043876
Series name: None
Non-Null Count    Dtype 
--------------    ----- 
1043877 non-null  string
dtypes: string(1)
memory usage: 8.0 MB
None
----------------------------------------
count     1043877
unique      95204
top             в
freq        45673
dtype: object
----------------------------------------
<class 'pandas.Series'>
RangeIndex: 1043877 entries, 0 to 1043876


In [16]:
statistics(p2)

----------------------------------------
<class 'pandas.Series'>
RangeIndex: 5001 entries, 0 to 5000
Series name: None
Non-Null Count  Dtype
--------------  -----
5001 non-null   str  
dtypes: str(1)
memory usage: 39.2 KB
None
----------------------------------------
count                                                  5001
unique                                                 5001
top       в -е годы советский союз охватила лихорадка — ...
freq                                                      1
dtype: object
----------------------------------------
<class 'pandas.Series'>
RangeIndex: 1043791 entries, 0 to 1043790
Series name: None
Non-Null Count    Dtype 
--------------    ----- 
1043791 non-null  string
dtypes: string(1)
memory usage: 8.0 MB
None
----------------------------------------
count     1043791
unique      95204
top             в
freq        45669
dtype: object
----------------------------------------
<class 'pandas.Series'>
RangeIndex: 1043791 entries, 0 to 1043790


In [17]:
statistics(p3)

----------------------------------------
<class 'pandas.Series'>
RangeIndex: 5001 entries, 0 to 5000
Series name: None
Non-Null Count  Dtype
--------------  -----
5001 non-null   str  
dtypes: str(1)
memory usage: 39.2 KB
None
----------------------------------------
count                                                  5001
unique                                                 5001
top       в -е годы советский союз охватила лихорадка — ...
freq                                                      1
dtype: object
----------------------------------------
<class 'pandas.Series'>
RangeIndex: 1045907 entries, 0 to 1045906
Series name: None
Non-Null Count    Dtype 
--------------    ----- 
1045907 non-null  string
dtypes: string(1)
memory usage: 8.0 MB
None
----------------------------------------
count     1045907
unique      96786
top             в
freq        45688
dtype: object
----------------------------------------
<class 'pandas.Series'>
RangeIndex: 1045907 entries, 0 to 1045906


In [18]:
statistics(p4)

----------------------------------------
<class 'pandas.Series'>
RangeIndex: 5000 entries, 0 to 4999
Series name: text
Non-Null Count  Dtype
--------------  -----
5000 non-null   str  
dtypes: str(1)
memory usage: 39.2 KB
None
----------------------------------------
count                                                  5000
unique                                                 5000
top       в -е годы советский союз охватила лихорадка — ...
freq                                                      1
Name: text, dtype: object
----------------------------------------
<class 'pandas.Series'>
RangeIndex: 1043877 entries, 0 to 1043876
Series name: None
Non-Null Count    Dtype 
--------------    ----- 
1043877 non-null  string
dtypes: string(1)
memory usage: 8.0 MB
None
----------------------------------------
count     1043877
unique      95204
top             в
freq        45673
dtype: object
----------------------------------------
<class 'pandas.Series'>
RangeIndex: 1043877 entries, 0